# PROCESAMIENTO DE LOS DATOS DEL DATASET "credits"

Cambiamos la columna "id" por la columa de "Id_movie" del datset movie (join)

In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
cred=pd.read_csv('./Database/credits.csv',delimiter = ',',encoding = "utf-8")
movies=pd.read_csv('./Database/titles.csv',delimiter = ',',encoding = "utf-8")

In [3]:
cred=cred.rename(columns={"id": "Id_movie_viejo"})

In [4]:
movies.columns

Index(['Id_movie_viejo', 'title', 'Id_movie'], dtype='object')

In [5]:
cred=cred.merge(movies[["Id_movie_viejo", "Id_movie"]], left_on="Id_movie_viejo", right_on="Id_movie_viejo", how="left")
cred.drop(columns="Id_movie_viejo",inplace=True)
cred

,cast,crew,Id_movie
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",2
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",3
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",4
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",5
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",45432
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",45433
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",45434
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",45435


Convertimos los elementos de las columnas de str a list

In [6]:
cred["cast"]=cred["cast"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
cred["crew"]=cred["crew"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

Transformemos la columna cast 

Cramos un nuevo dataframe solo con la columna cast 

In [7]:
df=pd.DataFrame(cred["cast"])
df

,cast
0,"[{'cast_id': 14, 'character': 'Woody (voice)',..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c..."
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah..."
4,"[{'cast_id': 1, 'character': 'George Banks', '..."
...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ..."
45472,"[{'cast_id': 1002, 'character': 'Sister Angela..."
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr..."
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ..."


Desanidamos la columna cast

In [8]:
df=pd.concat([df.drop(columns="cast"), pd.json_normalize(df["cast"])], axis=1)

C:\Users\maria\AppData\Local\Temp\ipykernel_24444\3172029258.py:1: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df=pd.concat([df.drop(columns="cast"), pd.json_normalize(df["cast"])], axis=1)


Cambiamos los nombres de las columnas desanidadas a col0, col1, etc 

In [9]:
df.columns=[f"col{i}" for i in range(df.shape[1])]

Convertimos los valores nulos a diccionarios vacíos

In [10]:
df=df.map(lambda x: x if isinstance(x, dict) else {})

Queremos hacer que la columna cast quede solo como listas de los actores. Por lo que debemos sacar de los diccionarios solo el nombre del actor 

In [11]:
df.loc[0,"col0"]

{'cast_id': 14,
 'character': 'Woody (voice)',
 'credit_id': '52fe4284c3a36847f8024f95',
 'gender': 2,
 'id': 31,
 'name': 'Tom Hanks',
 'order': 0,
 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}

Guarda en una lista de listas los nombres de los actores. Cada lista de adentro corresponde a cada película. Luego con eso crea una nueva columna 

In [12]:
lista_actores=[]
for i in range(df.shape[1]):
    df_normalizado=pd.json_normalize(df[f"col{i}"])
    lista_actores.append(df_normalizado["name"].rename(f"actor_{i}"))

actores_df=pd.concat(lista_actores, axis=1)
df["actors"]=actores_df.apply(lambda row: row.tolist(), axis=1)


Eliminamos las columnas "col"

In [13]:
df.drop(columns=[f"col{i}" for i in range(df.shape[1]-1)], inplace=True)

In [14]:
df

,actors
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,"[Steve Martin, Diane Keaton, Martin Short, Kim..."
...,...
45471,"[Leila Hatami, Kourosh Tahami, Elham Korda, na..."
45472,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe..."
45473,"[Erika Eleniak, Adam Baldwin, Julie du Page, J..."
45474,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa..."


Concatenamos la columna actors con dataframe cred y eliminamos la columna cast

In [15]:
cred.drop(columns="cast",inplace=True)
cred1=pd.concat([cred,df],axis=1)
cred1

,crew,Id_movie,actors
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",2,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",3,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,"[{'credit_id': '52fe44779251416c91011acb', 'de...",4,"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",5,"[Steve Martin, Diane Keaton, Martin Short, Kim..."
...,...,...,...
45471,"[{'credit_id': '5894a97d925141426c00818c', 'de...",45432,"[Leila Hatami, Kourosh Tahami, Elham Korda, na..."
45472,"[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",45433,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe..."
45473,"[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",45434,"[Erika Eleniak, Adam Baldwin, Julie du Page, J..."
45474,"[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",45435,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa..."


Transformamos la columna crew

Creamos un dataframe solo con la columna crew

In [16]:
df1=pd.DataFrame(cred1["crew"])

In [17]:
df1

,crew
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,"[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,"[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...
45471,"[{'credit_id': '5894a97d925141426c00818c', 'de..."
45472,"[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de..."
45473,"[{'credit_id': '52fe4776c3a368484e0c8387', 'de..."
45474,"[{'credit_id': '533bccebc3a36844cf0011a7', 'de..."


In [18]:
df1.loc[0,"crew"][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

Primero chequeemos que no hayan listas vacías 

In [19]:
for i in range(1):
        if len(df1.loc[i,"crew"])==0:
            print(f"fila {i}")
            print(len(df1.loc[i,"crew"]))
            for j in range(len(df1.loc[i,"crew"])):
                print(j)
        else:
             pass


In [20]:
df2=df1

En este caso solo nos queremos quedar, en cada fila, con los diccionarios, dentro de las listas, que tengan la clave "job" : "Director". Al resto los eliminamos de las listas

In [21]:
for i in range(df2.shape[0]):
    if len(df2.loc[i,"crew"])!=0:
        for j in range(len(df2.loc[i,"crew"])-1,-1,-1):
            if df2.loc[i,"crew"][j]["job"]!="Director":
                del df2.loc[i,"crew"][j]
            else:
                pass
    else:
        pass

Desanidamos las listas

In [22]:
df2=pd.concat([df2.drop(columns="crew"), pd.json_normalize(df2["crew"])], axis=1)

C:\Users\maria\AppData\Local\Temp\ipykernel_24444\2515343222.py:1: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df2=pd.concat([df2.drop(columns="crew"), pd.json_normalize(df2["crew"])], axis=1)


Cambiamos los nombres de las columnas y llenamos los valores nulos con diccionarios vacíos 

In [23]:
df2.columns=[f"col{i}" for i in range(df2.shape[1])]
df2=df2.applymap(lambda x: x if isinstance(x, dict) else {})

C:\Users\maria\AppData\Local\Temp\ipykernel_24444\53644541.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2=df2.applymap(lambda x: x if isinstance(x, dict) else {})


Guarda en una lista de listas los nombres de los directores. Cada lista de adentro corresponde a cada película. Luego con eso crea una nueva columna 

In [24]:
lista_directores=[]
for i in range(df2.shape[1]):
    df2_normalizado=pd.json_normalize(df2[f"col{i}"])
    lista_directores.append(df2_normalizado["name"].rename(f"director_{i}"))

directores_df=pd.concat(lista_directores, axis=1)
df2["directors"]=directores_df.apply(lambda row: row.tolist(), axis=1)

In [25]:
df2.drop(columns=[f"col{i}" for i in range(df2.shape[1]-1)], inplace=True)

In [26]:
df2

,directors
0,"[John Lasseter, nan, nan, nan, nan, nan, nan, ..."
1,"[Joe Johnston, nan, nan, nan, nan, nan, nan, n..."
2,"[Howard Deutch, nan, nan, nan, nan, nan, nan, ..."
3,"[Forest Whitaker, nan, nan, nan, nan, nan, nan..."
4,"[Charles Shyer, nan, nan, nan, nan, nan, nan, ..."
...,...
45471,"[Hamid Nematollah, nan, nan, nan, nan, nan, na..."
45472,"[Lav Diaz, nan, nan, nan, nan, nan, nan, nan, ..."
45473,"[Mark L. Lester, nan, nan, nan, nan, nan, nan,..."
45474,"[Yakov Protazanov, nan, nan, nan, nan, nan, na..."


Concatenamos la columna derectors con el dataframe cred y eliminamos la columna crew

In [27]:
cred1.drop(columns="crew",inplace=True)
cred2=pd.concat([cred1,df2],axis=1)
cred2

,Id_movie,actors,directors
0,1,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[John Lasseter, nan, nan, nan, nan, nan, nan, ..."
1,2,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Joe Johnston, nan, nan, nan, nan, nan, nan, n..."
2,3,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Howard Deutch, nan, nan, nan, nan, nan, nan, ..."
3,4,"[Whitney Houston, Angela Bassett, Loretta Devi...","[Forest Whitaker, nan, nan, nan, nan, nan, nan..."
4,5,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[Charles Shyer, nan, nan, nan, nan, nan, nan, ..."
...,...,...,...
45471,45432,"[Leila Hatami, Kourosh Tahami, Elham Korda, na...","[Hamid Nematollah, nan, nan, nan, nan, nan, na..."
45472,45433,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...","[Lav Diaz, nan, nan, nan, nan, nan, nan, nan, ..."
45473,45434,"[Erika Eleniak, Adam Baldwin, Julie du Page, J...","[Mark L. Lester, nan, nan, nan, nan, nan, nan,..."
45474,45435,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...","[Yakov Protazanov, nan, nan, nan, nan, nan, na..."


Elimino los elementos nan de las listas 

In [28]:
for i in cred2.columns:
    cred2[i] = cred2[i].apply(lambda x: [j for j in x if not pd.isna(j)] if isinstance(x, list) else x)

In [29]:
cred2

,Id_movie,actors,directors
0,1,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter]
1,2,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston]
2,3,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",[Howard Deutch]
3,4,"[Whitney Houston, Angela Bassett, Loretta Devi...",[Forest Whitaker]
4,5,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Charles Shyer]
...,...,...,...
45471,45432,"[Leila Hatami, Kourosh Tahami, Elham Korda]",[Hamid Nematollah]
45472,45433,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...",[Lav Diaz]
45473,45434,"[Erika Eleniak, Adam Baldwin, Julie du Page, J...",[Mark L. Lester]
45474,45435,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",[Yakov Protazanov]


Eliminamos los valores repetidos de la columna Id_movie

In [30]:
cred3=cred2.drop_duplicates(subset="Id_movie",ignore_index=True)
cred3.shape

(45432, 3)

Queremos quedarnos solo con las películas que estén en el dataset movies_normalizado. Para esto traemos la columna return con un merge y eliminamos los valores nulos de esta columna.

In [31]:
movie=pd.read_csv('./Database/movies_normalizado.csv',delimiter = ',',encoding = "utf-8")
cred4=cred3.merge(movie[["Id_movie", "return"]], left_on="Id_movie", right_on="Id_movie", how="left")

In [32]:
cred4=cred4.dropna(subset="return")
cred4.info()
cred4.drop(columns="return",inplace=True)
cred4.to_csv('./Database/credits_normalizado.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 45345 entries, 0 to 45431
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Id_movie   45345 non-null  int64  
 1   actors     45345 non-null  object 
 2   directors  45345 non-null  object 
 3   return     45345 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.7+ MB
